# 抽象基底クラスの定義およびOMMX Adapterの実装ガイド

## 抽象基底クラスの定義

抽象基底クラス `SolverAdapter` は、OMMXからSolverを利用するためのAdapterのインターフェイスを規格化するためのものである。

```python
from abc import ABC, abstractmethod
from typing import Any
from ommx.v1 import Instance, Solution

SolverInput = Any
SolverOutput = Any

class SolverAdapter(ABC):
    """
    OMMXからSolverを利用するためのAdapterの抽象基底クラスです。
    詳しいアダプターの実装方法は (TODO: ドキュメントへのリンク) を参照してください。
    """

    @abstractmethod
    def __init__(self, ommx_instance: Instance):
        pass

    @staticmethod
    @abstractmethod
    def solve(ommx_instance: Instance) -> Solution:
        pass

    @property
    @abstractmethod
    def solver_input(self) -> SolverInput:
        pass

    @abstractmethod
    def decode(self, data: SolverOutput) -> Solution:
        pass
```

# OMMX Adapter実装ガイド

本ドキュメントは、OMMXからSolverを実行するためのAdapterを実装する際の推奨事項を記載したものである。下記の内容に従いOMMX Adapterを実装することで、他のOMMX Adapterと一定程度の一貫性を保つことができ、複数のOMMX Adapterを利用するユーザーにとってよりよいユーザー体験の提供に繋がる。そのため、OMMXコミュニティとしては以下の内容に従うことを推奨したい。

## 推奨事項

- OMMXからSolverを利用するためのアダプター `OMMXxxxAdapter`は、抽象基底クラス `SolverAdapter`を継承して実装することを推奨する
- `SolverAdapter`に実装されているメソッドの引数以外のものを引数として定義する場合は、キーワード限定引数を利用し、デフォルト値を設定するものとする
- `OMMXxxxAdapter.solve` メソッドは、OMMXからSolverを簡単に使うための静的メソッドとして提供されるべきである
    - Solverのパラメーターを全て引数として実装する必要はない。そのようなユースケースは `OMMXxxxAdapter` クラスを直接使うことで実現できれば十分である
- `OMMXxxxAdapter` のコンストラクタ内で、以下の処理を行うようにすることを推奨する
    - `ommx.v1.Instance` からSolverの入力形式（データやオブジェクト）を生成する処理
    - `OMMXxxxAdapter.decode` に必要となるデータの生成する処理
- `OMMXxxxAdapter.solver_input` プロパティは、OMMXからSolverの入力形式（データやオブジェクト）を取得するプロパティとして提供されるべきである
- `OMMXxxxAdapter.decode` メソッドは、Solverの出力形式（データやオブジェクト）を `ommx.v1.Instance` へと変換するメソッドとして提供されるべきである

## 検討事項

以下では、推奨するほどではないが実装を検討すると良いものを列挙する。

- `OMMXxxxAdapter.decode_to_state` メソッド: Solverの出力形式（データやオブジェクト）を `ommx.v1.State` に変換するためのメソッド
    
    ```python
    class OMMXxxxAdapter(SolverAdapter):
    		...
    		def decode_to_state(self, data: SolverOutput) -> ommx.v1.State:
    			...
    ```
    
- `yyy_to_instance` 関数: Solverの入力形式を `ommx.v1.Instance` に変換する関数
    
    ```python
    def xxx_to_instance(data: Any) -> ommmx.v1.Instance:
    		...
    ```
    

## 推奨事項を満たすOMMX Adapterが保証するユーザー体験

上記の推奨事項を満たすOMMX Adapterは、以下の一貫したユーザー体験を与えることができる。

- Solverを気軽に利用したいユーザー（Solverの入出力方法やパラメーターを把握したくないユーザー）は、共通で提供されている静的メソッド `solve` で簡単にSolverを実行できる
    
    ```python
    from ommx_xxx_adapter import OMMXxxxAdapter
    
    ommx_solution = OMMXxxxAdapter.solve(ommx_instance)
    ```
    
- Solverをチューニングして利用したいユーザーは、直接 `OMMXxxxAdapter` クラスを使うことでSolverのパラメーター等を設定して実行できる